# Setup

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime
from src import (get_data as gd, 
                 prepare as prep, 
                 aggregate_data as aggregate,
                 push_data,
                 sedi_rate,
                 age_sr_plot)
###
orig_dir = os.getcwd()

# Main Data

### Retrieve data 

In [2]:
dates = gd.AgeFromFileOneCore()

FileChooser(path='/home/jovyan/work/input_files', filename='', title='', show_hidden=False, select_desc='Selec…

In [3]:
dates.get_dates()

### Add calibration curve to data

In [4]:
dates.select_calibration_curve(default_curve = 'IntCal20', user_selection = False)

In [5]:
dates.add_calibration_curve()

### Put data in variables

In [6]:
AllAges = dates.all_ages
CoreIDs = dates.all_coreid_list
CoreLengths = dates.all_core_lengths
SR_mode = 'naive'

In [7]:
aggRC = lambda: None
aggRC.dttp = "No"

# Reservoir correction

#### Prep

In [16]:
os.chdir(orig_dir)
RC = prep.PrepForReservoirCorrection(all_ages = AllAges)
RC.prep_it()
RC_Frame = RC.RC_Frame
RC_CoreIDs = RC.RC_CoreIDs
RC_CoreLengths = CoreLengths[CoreLengths.coreid.isin(RC_CoreIDs)]

#### R run

In [17]:
%get RC_Frame
%get RC_CoreIDs 
%get RC_CoreLengths

In [18]:
K_fine <- 100
source('src/Run_DefineReservoir.R', local = TRUE)

Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Done with core number: 1 out of 1



In [19]:
%put reservoir_core_results

#### Aggregate results

In [20]:
aggRC = aggregate.AggDataReservoir(results = reservoir_core_results,
                                   surface_dates = RC.desired_surface_dates,
                                   verbose = 0)
aggRC.results_agg()

In [21]:
AllAges = aggRC.add_reservoir(all_ages = AllAges)

# Calibration
#### Prep

In [22]:
calib = prep.PrepForCalibration(all_ages = AllAges)
calib.prep_it()
calib_Frame = calib.calib_Frame

#### R run

In [23]:
%get calib_Frame --from SoS

In [24]:
source('src/Run_calibration.R', local = TRUE)

In [25]:
%put calib_dates

# Age Modeling
### UNDATABLE
#### Prep

In [26]:
Undatable = prep.PrepForUndatable(all_ages = AllAges, 
                                  all_coreid_list = CoreIDs)
Undatable.prep_it()
CoreID_array = Undatable.CoreID_array

Information: New files for Undatable created!


#### Octave run

In [27]:
%get CoreID_array

In [28]:
xfactor = 0.1;
bootpc = 30;
run_Undatable(CoreID_array, xfactor, bootpc);

#### Aggregate results

In [29]:
aggU = aggregate.AggDataUndatable(Undatable, orig_dir, dttp = aggRC.dttp)
aggU.results_agg()

#### Calculate sedimentation rate

In [30]:
SRUn = sedi_rate.CalculateSediRate(aggU, model = 'Undatable', coreid = CoreIDs, mode = SR_mode)
SRUn.calculating_SR()

#### Push results to database

In [31]:
push = push_data.PushIt(agg = aggU,
                        engine = dates.engine,
                        model = 'Undatable')
push.push_to_db()

Information: LANDO is not connected to a database, so nothing was uploaded.


In [32]:
os.chdir(orig_dir)
push.delete_files(location_UndatableFolder = Undatable.location_UndatableFolder,
                  coreids = Undatable.coreid_df)

Information: All unwanted Undatable files have been deleted


### Bchron

#### Prep

In [33]:
os.chdir(orig_dir)
Bchron = prep.PrepForBchron(all_ages = AllAges)
Bchron.prep_it()
Bchron_Frame = Bchron.Bchron_Frame

#### R run

In [34]:
%get Bchron_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [35]:
source('src/Run_Bchron.R')

Running Bchronology...
  |============================================================| 100%
Run completed!


 Done with core EN18218 - Number 1 out of 1



In [36]:
%put Bchron_core_results

#### Aggregate results

In [37]:
aggBc = aggregate.AggDataBchron(Bchron_core_results, dttp = aggRC.dttp)
aggBc.results_agg()

#### Calculating sedimentation rate

In [38]:
SRBc = sedi_rate.CalculateSediRate(aggBc, model = 'Bchron', coreid = CoreIDs, mode = SR_mode)
SRBc.calculating_SR()

#### Push results to database

In [39]:
push = push_data.PushIt(agg = aggBc,
                        engine = dates.engine,
                        model = 'Bchron')
push.push_to_db()

Information: LANDO is not connected to a database, so nothing was uploaded.


### hamstr

#### Prep

In [40]:
hamstr = prep.PrepForHamstr(all_ages = AllAges)
hamstr.prep_it()
hamstr_Frame = hamstr.hamstr_Frame

#### R Run

In [41]:
%get hamstr_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [42]:
K_fine <- 100
source('src/Run_hamstr.R', local = TRUE)

Done with core: EN18218



In [43]:
%put hamstr_core_results

#### Aggregate

In [44]:
aggh = aggregate.AggDataHamstr(hamstr_core_results, dttp = aggRC.dttp)
aggh.results_agg()

#### Calculating sedimentation rate

In [45]:
SRh = sedi_rate.CalculateSediRate(aggh, model = 'hamstr', coreid = CoreIDs, mode = SR_mode)
SRh.calculating_SR()

#### Push results to database

In [46]:
push = push_data.PushIt(agg = aggh,
                        engine = dates.engine,
                        model = 'hamstr')
push.push_to_db()

Information: LANDO is not connected to a database, so nothing was uploaded.


### Bacon

#### Prep

In [8]:
Bacon = prep.PrepForBacon(all_ages = AllAges)
Bacon.prep_it()
Bacon_Frame = Bacon.Bacon_Frame

#### R Run

In [9]:
%get Bacon_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [10]:
remotes::install_github("GPawi/hamstrbacon", 
                        upgrade = 'never', 
                       force = TRUE,
                        build = TRUE,
                       lib = "/opt/conda/lib/R/library")


* installing *source* package ‘hamstrbacon’ ...
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpPmGAuX/remotesd36ec31395/GPawi-hamstrbacon-04d304a/DESCRIPTION’ ... OK
* preparing ‘hamstrbacon’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Removed empty directory ‘hamstrbacon/vignettes’
* building ‘hamstrbacon_0.1.1.tar.gz’



** testing if installed package keeps a record of temporary installation path
* DONE (hamstrbacon)


In [ ]:
acc.shape <- 1.5
acc.mean <- 20
mem.strength <- 10
mem.mean <- 0.5
ssize <- 2000
source('src/Run_Bacon.R', local = TRUE)

Running Bacon for core EN18208 (attempt 1, ssize = 2000)



The run's files will be put in this folder: /tmp/RtmpPmGAuX


Warning, I removed quotation marks

Adapting acc.mean to 50 yr/cm

Warning message in file(con, "r"):
“file("") only supports open = "w+" and open = "w+b": using the former”


In [ ]:
%put Bacon_core_results

#### Aggregate

In [ ]:
aggBa = aggregate.AggDataBacon(Bacon_core_results, dttp = aggRC.dttp)
aggBa.results_agg()

#### Calculating sedimentation rate

In [ ]:
SRBa = sedi_rate.CalculateSediRate(aggBa, model = 'Bacon', coreid = CoreIDs, mode = SR_mode)
SRBa.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggBa,
                        engine = dates.engine,
                        model = 'Bacon')
push.push_to_db()

### Clam

#### Prep

In [8]:
clam = prep.PrepForClam(all_ages = AllAges)
clam.prep_it()
clam_Frame = clam.clam_Frame

#### R Run

In [9]:
%get clam_Frame --from SoS
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [10]:
types_curve <- 1:5
smoothness_curve <- 0.1*(1:10)
poly_degree_curve <- 1:4
best_fit <- TRUE

In [22]:
source('src/Run_clam.R', local = TRUE)

⚠️  No output for core EN18218, type 1
⚠️  No output for core EN18218, type 2
⚠️  No output for core EN18218, type 2
⚠️  No output for core EN18218, type 2
⚠️  No output for core EN18218, type 2
⚠️  No output for core EN18218, type 3
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 4
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN18218, type 5
⚠️  No output for core EN

Warning message:
“Unknown or uninitialised column: `fit`.”
Warning message:
“Unknown or uninitialised column: `fit`.”
Warning message in min(result_types$fit):
“no non-missing arguments to min; returning Inf”


In [12]:
%put clam_core_results

#### Aggregate

In [ ]:
aggcl = aggregate.AggDataClam(clam_core_results, dttp = aggRC.dttp)
aggcl.results_agg()

#### Calculating sedimentation rate

In [ ]:
SRcl = sedi_rate.CalculateSediRate(aggcl, model = 'clam', coreid = CoreIDs, mode = SR_mode)
SRcl.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggcl,
                        engine = dates.engine,
                        model = 'clam')
push.push_to_db()

# Plotting

In [ ]:
plot_data = {'Undatable' : [aggU.age_model_result_Undatable, SRUn.SR_model_result_Undatable],
             'Bchron': [aggBc.age_model_result_Bchron, SRBc.SR_model_result_Bchron],
             'hamstr': [aggh.age_model_result_hamstr, SRh.SR_model_result_hamstr],
             'Bacon': [aggBa.age_model_result_Bacon, SRBa.SR_model_result_Bacon],
             'clam': [aggcl.age_model_result_clam, SRcl.SR_model_result_clam],
             'calib_dates': calib_dates}

In [ ]:
ASRplot = age_sr_plot.PlotAgeSR(plot_data, coreid = CoreIDs, dttp = aggRC.dttp)

In [ ]:
ASRplot.plot_graph(orig_dir = orig_dir, sigma_range = 'both',
                   bin_size = 1000, xlim_max = 21000, number_col = 8, reduce_plot_axis = True,
                   only_combined = True, save = True, for_color_blind = False, as_jpg = False)

# Optimization

In [ ]:
proxy = gd.ProxyFromFile()

In [ ]:
proxy_ts = proxy.get_proxy()
###
dict_model_name = ASRplot.dict_model_name
dict_SR_median_age = ASRplot.dict_SR_median_age

In [ ]:
%get dict_SR_median_age --from SoS
%get dict_model_name --from SoS
%get proxy_ts --from SoS

In [ ]:
curr_nreps     <- 1000
curr_minseg    <- 50
curr_cptmethod <- "mean+variance"

In [ ]:
source('src/Run_Optimization.R', local = TRUE)

In [ ]:
%put result_list 
%put fitting_values

In [ ]:
optimizedASRplot = age_sr_plot.PlotAgeSR(plot_data, coreid = CoreIDs, dttp = aggRC.dttp)

In [ ]:
optimizedASRplot.plot_optimized_graph(optimization_values = result_list, fitting_values = fitting_values, 
                                      proxy = proxy.name, proxy_data = proxy_ts, orig_dir = orig_dir, 
                                      sigma_range = 'both', inclusion_threshold = 0.03, show_fitting_models = True,
                                      bin_size = 1000, xlim_max = 21000, number_col = 8, reduce_plot_axis = True,
                                      only_combined = True, save = True, for_color_blind = False, as_jpg = False)